# CNN model

In [1]:
# Imports

import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Local test setup
tf.config.threading.set_inter_op_parallelism_threads(6) # 8
tf.config.threading.set_intra_op_parallelism_threads(14) # 24

### 1. Data Preprocessing

In [ ]:
# Loading CIFAR-10 dataset

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('Training set shape:', x_train.shape)
print('Testing set shape:', x_test.shape)

print('Training labels shape:', y_train.shape)
print('Testing labels shape:', y_test.shape)

labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

print(labels)

In [ ]:
# Visualizing some images

plt.figure(figsize=(8, 8))

for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_train[i])
    plt.xlabel(labels[y_train[i][0]])

plt.show()

In [4]:
# One-hot encoding the labels

y_train = tf.keras.utils.to_categorical(y_train, num_classes = 10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes = 10)

In [ ]:
# Normalizing the Images

x_train = x_train / 255
x_test = x_test / 255

print(x_train[0].min())
print(x_train[0].max())

In [ ]:
# Resizing images

x_train = tf.image.resize(x_train, (64, 64))
x_test = tf.image.resize(x_test, (64, 64))

print('Resized training image size:', x_train[0].shape)
print('Resized testing image size:', x_test[0].shape)

In [7]:
# Augmenting data

# TODO

### 2. Model Architecture

In [ ]:
# Defining the model

model = Sequential()

model.add(Conv2D(32, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', input_shape = (64, 64, 3)))
model.add(Conv2D(32, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(Conv2D(64, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(128, activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(Dropout(0.5))

model.add(Dense(64, activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(Dropout(0.5))

model.add(Dense(10, activation = 'softmax'))

# Model summary

model.summary()

### 3. Model Training

In [ ]:
# Compiling the model

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# Early stopping

stopping = EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)

# Training the model

model.fit(x_train, y_train, batch_size = 64, epochs = 50, validation_split = 0.1, callbacks = [stopping])

# Evaluating the model on the test data

test_loss, test_accuracy = model.evaluate(x_test, y_test)

print('Test data loss:', test_loss)
print('Test data accuracy:', test_accuracy)